In [12]:
%matplotlib inline
import numpy as np
import openmc
import openmc.model

In [15]:
def inf_model(fuel_depth, ref_depth, fuel_thickness, cyl_rad, mod_thickness):
    
    #fuel
    fuel = openmc.Material(name="Am242m")
    fuel.add_nuclide('Am242_m1',1)
    fuel.set_density('g/cm3',13.6)
    
    #Be moderator
    Be = openmc.Material(name="Be")
    Be.add_nuclide("Be9", 1.0)
    Be.set_density("g/cm3", 1.85)
    
    #"vacuum"
    vac = openmc.Material(name="H1")
    vac.add_nuclide('H1',1.0)
    vac.set_density('g/cm3',0.000001)
    
    # Outer radius of fuel
    cyl_inner = cyl_rad - fuel_thickness #inner radius for 2micron fuel layer

    #edge length of hexagonal fuel cell
    hex_dis = cyl_rad + mod_thickness
    edge = hex_dis * 2 / np.sqrt(3)

    #outer radius of fuel disk
    disk_outer = 50 # radius in cm
    
    #define a cell surface for a single fuel cylinder
    f_inner_surf = openmc.ZCylinder(R=cyl_rad-fuel_thickness, boundary_type="transmission")
    f_outer_surf = openmc.ZCylinder(R=cyl_rad, boundary_type="transmission")
    f_lower_surf = openmc.ZPlane(z0=0, boundary_type="vacuum")
    f_upper_surf = openmc.ZPlane(z0=fuel_depth, boundary_type="transmission")
    m_upper_surf = openmc.ZPlane(z0=fuel_depth+ref_depth, boundary_type="vacuum")
    periodic_hex = openmc.get_hexagonal_prism(edge_length=edge, orientation='x', boundary_type="periodic")

    fuel_region = -f_outer_surf & +f_inner_surf & -f_upper_surf & +f_lower_surf
    vac_region = -f_inner_surf & +f_lower_surf & -f_upper_surf
    mod_region_per = (+f_outer_surf & -f_upper_surf & + f_lower_surf & periodic_hex) | (+f_upper_surf & -m_upper_surf & periodic_hex)

    #fill regions with materials
    f=openmc.Cell(1, 'fuel')
    f.fill=fuel
    f.region=fuel_region
    
    m_per = openmc.Cell(2, 'moderator region with periodic outer bounds')
    m_per.fill=Be
    m_per.region=mod_region_per

    v = openmc.Cell(5, 'vacuum')
    v.fill = vac
    v.region = vac_region
    
    #create a root universe
    uni = openmc.Universe(cells=[f, m_per, v])
    
    #create root universe
    geom = openmc.Geometry()
    geom.root_universe = uni

    #configure statistics geometry
    #source = openmc.stats.Box((-20, -20, 0), (20, 20, fuel_depth))
    source = openmc.stats.Point((cyl_inner+fuel_thickness/2, 0, fuel_depth/2))
    src = openmc.Source(space=source)
    
    #create materials
    mats = openmc.Materials()
    mats.cross_sections = '/home/james/nndc_hdf5/cross_sections.xml' #comment this out/replace if you need to
    mats.append(fuel)
    mats += [Be]
    mats += [vac]
    
    #configure run settings
    settings = openmc.Settings()
    settings.particles = 1000
    settings.batches = 300
    settings.inactive = 20
    settings.source = src
    
    settings.output = {'tallies': False}
    
    model = openmc.model.Model(geom, mats, settings)
    
    return model

In [ ]:
# Perform the search
crit, guesses, keffs = openmc.search_for_keff(inf_model, bracket=[25., 35.],
                                                  model_args = {
                                                      'ref_depth':10,
                                                      'fuel_thickness':1e-3,
                                                      'cyl_rad':1,
                                                      'mod_thickness':6},
                                                  tol=1.E-2, bracketed_method='bisect',
                                                  print_iterations=True)

print('Critical fuel depth: {:4.0f}'.format(crit))

/home/james/anaconda3/lib/python3.6/site-packages/openmc/mixin.py:61: IDWarning: Another Cell instance already exists with id=1.
  warn(msg, IDWarning)
/home/james/anaconda3/lib/python3.6/site-packages/openmc/mixin.py:61: IDWarning: Another Cell instance already exists with id=2.
  warn(msg, IDWarning)
/home/james/anaconda3/lib/python3.6/site-packages/openmc/mixin.py:61: IDWarning: Another Cell instance already exists with id=5.
  warn(msg, IDWarning)


Iteration: 1; Guess of 2.50e+01 produced a keff of 0.79949 +/- 0.00257


/home/james/anaconda3/lib/python3.6/site-packages/openmc/mixin.py:61: IDWarning: Another Cell instance already exists with id=1.
  warn(msg, IDWarning)
/home/james/anaconda3/lib/python3.6/site-packages/openmc/mixin.py:61: IDWarning: Another Cell instance already exists with id=2.
  warn(msg, IDWarning)
/home/james/anaconda3/lib/python3.6/site-packages/openmc/mixin.py:61: IDWarning: Another Cell instance already exists with id=5.
  warn(msg, IDWarning)


Iteration: 2; Guess of 3.50e+01 produced a keff of 1.18179 +/- 0.00284


/home/james/anaconda3/lib/python3.6/site-packages/openmc/mixin.py:61: IDWarning: Another Cell instance already exists with id=1.
  warn(msg, IDWarning)
/home/james/anaconda3/lib/python3.6/site-packages/openmc/mixin.py:61: IDWarning: Another Cell instance already exists with id=2.
  warn(msg, IDWarning)
/home/james/anaconda3/lib/python3.6/site-packages/openmc/mixin.py:61: IDWarning: Another Cell instance already exists with id=5.
  warn(msg, IDWarning)


Iteration: 3; Guess of 3.00e+01 produced a keff of 1.00767 +/- 0.00282


/home/james/anaconda3/lib/python3.6/site-packages/openmc/mixin.py:61: IDWarning: Another Cell instance already exists with id=1.
  warn(msg, IDWarning)
/home/james/anaconda3/lib/python3.6/site-packages/openmc/mixin.py:61: IDWarning: Another Cell instance already exists with id=2.
  warn(msg, IDWarning)
/home/james/anaconda3/lib/python3.6/site-packages/openmc/mixin.py:61: IDWarning: Another Cell instance already exists with id=5.
  warn(msg, IDWarning)
